In [ ]:
import sys
import xarray as xr
import numpy as np
import h5py

## dineof rmse

In [ ]:
# rmse
def rmse_DINEOF(p):
    # open files
    ds_orig = xr.open_dataset('Enki_LLC_orig.nc')
    ds_recon = xr.open_dataset('Enki_LLC_DINEOF_p{}.nc'.format(p))
    mask_file = 'mae_mask_t75_p10.h5'
    f_ma = h5py.File(mask_file, 'r')
    
    # extract data
    orig_imgs = np.asarray(ds_orig.variables['SST'])
    recon_imgs = np.asarray(ds_recon.variables['sst_filled'])
    mask_imgs = []
    for i in range(180):
        mask_imgs.append(f_ma['valid'][i,0,...])
        
    print('calculate')
    calc = (recon_imgs - orig_imgs)*mask_imgs
    calc = calc**2
    nmask = np.sum(mask_imgs, axis=(1,2))
    calc = np.sum(calc, axis=(1,2)) / nmask
    rmse = np.sqrt(calc)
    
    return rmse


In [ ]:
# do 
mask_p = [10,20,30,40,50]
for p in mask_p:
    rmse = rmse_DINEOF(p)

## creating the cloudy dineof files

In [ ]:
ds_orig = xr.open_dataset("Enki_LLC_orig.nc")
for p in mask_p:
    # filenames
    mask_file = 'mae_mask_t75_p{}.h5'.format(p)
    output_file = 'Enki_LLC_p{}.nc'.format(p)
    
    # create copy xarray
    ds = ds_orig.copy(deep=True)
    
    # grab mask
    mask_img = []
    f_ma = h5py.File(mask_file, 'r')
    for i in range(180):
        mask_img.append(f_ma['valid'][i,0,...])
        
    # replace clouds with missing value = -9999
    mask_img = np.asarray(mask_img)
    rmask_img = (1 - mask_img)
    fillm = mask_img*(-9999)
    ds.SST.values = ds_orig.SST.values*rmask_img + fillm
    
    # create new .nc file
    ds.to_netcdf(path=output_file, encoding={'SST': {'_FillValue': -9999}})